In [ ]:
import pandas as pd
import networkx as nx

path = 'Data/'

In [ ]:
import pickle5 as pickle
import time

In [ ]:
df = pd.read_parquet('{}file-name_query03-05.parquet'.format(path))
df

In [ ]:
texts_df = pd.read_parquet('{}topics_embeddings_2.parquet'.format(path))
texts_df

In [ ]:
from bertopic import BERTopic

topic_model = BERTopic.load('{}topic_model03-05'.format(path))

In [ ]:
def make_networkx(df, texts_df, topic_model,
                    id_feature='doi', special_attributes={'name': 'smile'}, list_features=[
                        'bioActivity', 'molType', 'collectionSpecie', 'collectionSite', 'collectionType', 'molecularMass', 'monoisotropicMass', 'cLogP', 'tpsa', 
                        'numberOfLipinskiViolations', 'numberOfH_bondAcceptors', 'numberOfH_bondDonors', 'numberOfRotableBonds', 'molecularVolume', 'name'
                    ],
):
    def create_edge(value1, value2, group1, group2, node_from):            
        if value1 != '' and value2 != '':
            print (value1, value2, group1, group2, node_from)
            G.add_edge(value1, value2, edge_group=f'{group1}_{group2}')
            G.nodes[value1]['group'] = group1
            G.nodes[value2]['group'] = group2
            G.nodes[value2]['node_from'] = node_from
    
    def create_attribute(attribute_feature, feature_value, attribute_value):
        if attribute_value != '':
            try:
                G.nodes[feature_value][attribute_feature] = attribute_value
            except:
                print('attribute_feature: {}, feature_value: {}, attribute_value: {}'. format(attribute_feature, feature_value, attribute_value))

    G = nx.Graph()
    for _, row in df.iterrows():
        for feature in list_features:
            create_edge(row[id_feature], row[feature], id_feature, feature, 'nubbe')
            if feature in special_attributes:
                create_attribute(special_attributes[feature], row[feature], row[special_attributes[feature]])
    for _, row in texts_df.iterrows():
        for topic in row['topics']:
            create_edge(row[id_feature], topic_model.get_topic_info(topic)['Name'].iloc[0], id_feature, 'topic', 'pdf')
        try:
            G.nodes[row[id_feature]]['embedding'] = row['embedding']
            G.nodes[row[id_feature]]['embedding_2'] = row['embedding_2']
            
        except:
            print('doi {} has no connections'.format(row[id_feature]))

    return G

G = make_networkx(df, texts_df, topic_model)

In [ ]:
!pip show networkx

In [ ]:
# if networkx is 3.0 version, !pip install networkx==2.8.8 . may be a kernel restart is required

In [ ]:
nx.write_gpickle(G, "{}hin03-05.gpickle".format(path))